In [1]:
import pandas as pd
from scipy.stats import chisquare

In [2]:
def get_years_dfs(misinfotext_path: str, fakespeak_path: str):
    misinfotext_file = pd.ExcelFile(misinfotext_path)
    
    for sheet in misinfotext_file.sheet_names:
        df = pd.read_excel(misinfotext_file, sheet_name=sheet)
        df["year"] = sheet
        yield df
    
    fakespeak_file = pd.ExcelFile(fakespeak_path)
    
    for sheet in fakespeak_file.sheet_names:
        df = pd.read_excel(fakespeak_file, sheet_name=sheet)
        df["year"] = sheet
        yield df

In [3]:
def build_chisquare_table(df: pd.DataFrame, ngrams: list[str]):
    # Fill in 0 if ngram doesn't exist in this df
    counts = df.set_index("ngram_text")["ngram_count"]\
                .reindex(ngrams, fill_value=0)

    table = pd.DataFrame({
        "ngram_text": ngrams,
        "ngram_count": counts.values
    })

    table["year"] = df["year"].iloc[0]

    return table

In [4]:
def run_stats_test(misinfotext_path: str, fakespeak_path: str, min_significant_pvalue=0.05):
    years_dfs = list(get_years_dfs(misinfotext_path, fakespeak_path))

    for df in years_dfs:
        df.sort_values("ngram_text", inplace=True)

    all_ngrams = sorted(set([
        ngram 
        for df in years_dfs
        for ngram in df["ngram_text"] 
    ]))

    tables = [build_chisquare_table(df, all_ngrams) for df in years_dfs]

    chisquare_result = chisquare([df["ngram_count"] for df in tables])

    num_insignificant_ngrams = sum(
        1 for pvalue in chisquare_result.pvalue 
        if pvalue > min_significant_pvalue
    )
    num_significant_ngrams = len(all_ngrams) - num_insignificant_ngrams

    print("There are", num_insignificant_ngrams, "ngrams that did not change significantly, and", 
          num_significant_ngrams, "ngrams that changed significantly")

    return pd.DataFrame({
        "ngram_text": all_ngrams,
        "statistic": chisquare_result.statistic,
        "pvalue": chisquare_result.pvalue
    })

In [5]:
results_all_types = run_stats_test(    
    misinfotext_path="./data/MisInfoText/Analysis_output/MisInfoText_ngrams.xlsx",
    fakespeak_path="./data/Fakespeak-ENG/Analysis_output/Fakespeak_ngrams.xlsx"
)
results_all_types

There are 0 ngrams that did not change significantly, and 1217 ngrams that changed significantly


,ngram_text,statistic,pvalue
0,000,910.192012,2.323198e-184
1,09 brown voted yea,60.000000,2.522085e-07
2,09 brown voted yea brown,60.000000,2.522085e-07
3,10,392.571429,2.514136e-74
4,10 000,240.000000,1.416373e-42
...,...,...,...
1212,young and somehow made it,45.000000,7.657266e-05
1213,young white men,105.000000,1.460740e-15
1214,your support the national media,45.000000,7.657266e-05
1215,yourself have son and daughter,45.000000,7.657266e-05


In [6]:
results_news_and_blog = run_stats_test(
    misinfotext_path="./data/MisInfoText/Analysis_output/news_and_blog/MisInfoText_news_and_blog_ngrams.xlsx",
    fakespeak_path="./data/Fakespeak-ENG/Analysis_output/news_and_blog/Fakespeak_news_and_blog_ngrams.xlsx"
)
results_news_and_blog

There are 0 ngrams that did not change significantly, and 1325 ngrams that changed significantly


,ngram_text,statistic,pvalue
0,000,762.666667,7.995255e-153
1,000 national guard troops,45.000000,7.657266e-05
2,10,283.200000,1.713702e-51
3,10 000,210.000000,1.959500e-36
4,10 000 national guard,45.000000,7.657266e-05
...,...,...,...
1320,young women avoid unwanted,30.000000,1.192150e-02
1321,your own membership active or,45.000000,7.657266e-05
1322,your support the national media,45.000000,7.657266e-05
1323,yourself have son and daughter,45.000000,7.657266e-05


In [7]:
results_press_release = run_stats_test(
    misinfotext_path="./data/MisInfoText/Analysis_output/press_release/MisInfoText_press_release_ngrams.xlsx",
    fakespeak_path="./data/Fakespeak-ENG/Analysis_output/press_release/Fakespeak_press_release_ngrams.xlsx"
)
results_press_release

There are 137 ngrams that did not change significantly, and 1512 ngrams that changed significantly


,ngram_text,statistic,pvalue
0,000,150.0,2.412498e-24
1,09 brown voted,75.0,5.662025e-10
2,09 brown voted yea,60.0,2.522085e-07
3,09 brown voted yea brown,60.0,2.522085e-07
4,10,210.0,1.959500e-36
...,...,...,...
1644,yucca mountain,90.0,9.918667e-13
1645,yucca mountain bill,30.0,1.192150e-02
1646,yucca mountain into permanent dump,15.0,4.514172e-01
1647,yucca mountain sen hellers record,15.0,4.514172e-01


In [8]:
results_social_media = run_stats_test(
    misinfotext_path="./data/MisInfoText/Analysis_output/social_media/MisInfoText_social_media_ngrams.xlsx",
    fakespeak_path="./data/Fakespeak-ENG/Analysis_output/social_media/Fakespeak_social_media_ngrams.xlsx"
)
results_social_media

There are 326 ngrams that did not change significantly, and 993 ngrams that changed significantly


,ngram_text,statistic,pvalue
0,000,481.258427,1.393978e-94
1,08,39.000000,1.999436e-04
2,10,39.000000,1.999436e-04
3,11 year old,52.000000,1.347814e-06
4,12,403.000000,5.198914e-78
...,...,...,...
1314,𝒃𝒍𝒐𝒐𝒅 𝒑𝒓𝒆𝒔𝒔𝒖𝒓𝒆 𝒘𝒊𝒍𝒍 𝒃𝒆,26.000000,1.700084e-02
1315,𝒑𝒓𝒆𝒔𝒔𝒖𝒓𝒆 𝒘𝒊𝒍𝒍 𝒃𝒆,26.000000,1.700084e-02
1316,𝗕𝗶𝗹𝗹 𝗮𝗻𝗱 𝗛𝗶𝗹𝗹𝗮𝗿𝘆,26.000000,1.700084e-02
1317,𝗕𝗶𝗹𝗹 𝗮𝗻𝗱 𝗛𝗶𝗹𝗹𝗮𝗿𝘆 𝗖𝗹𝗶𝗻𝘁𝗼𝗻,26.000000,1.700084e-02
